# Lab 03: TensorFlow vs. PyTorch




## TensorFlow Implementation

In [ ]:
# Installing tensorflow
pip install tensorflow

  Using cached protobuf-3.19.6-cp37-cp37m-win_amd64.whl.metadata (807 bytes)
Using cached protobuf-3.19.6-cp37-cp37m-win_amd64.whl (896 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.14.1 requires protobuf>=3.20.2, but you have protobuf 3.19.6 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Normalization factor
x_test = x_test / 255     # Normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(10, activation='softmax')  
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Using categorical_crossentropy loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Outputs the training time
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8666 - loss: 0.4910
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9557 - loss: 0.1518
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9698 - loss: 0.1046
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9765 - loss: 0.0808
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9811 - loss: 0.0653
TF Training time: 22.83 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9661 - loss: 0.1037


[0.09069951623678207, 0.9714000225067139]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Thanush\AppData\Local\Temp\tmpzcz53sjg\assets


INFO:tensorflow:Assets written to: C:\Users\Thanush\AppData\Local\Temp\tmpzcz53sjg\assets


Saved artifact at 'C:\Users\Thanush\AppData\Local\Temp\tmpzcz53sjg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1903041505104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1903041506064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1903041505488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1903041505872: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [ ]:
#Installing torch
!pip install torch

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB 2.0 MB/s eta 0:01:46
   ---------------------------------------- 0.1/216.1 MB 2.1 MB/s eta 0:01:41
   ---------------------------------------- 0.3/216.1 MB 2.6 MB/s eta 0:01:23
   ---------------------------------------- 0.5/216.1 MB 3.1 MB/s eta 0:01:10
   ---------------------------------------- 1.0/216.1 MB 4.7 MB/s eta 0:00:46
   ---------------------------------------- 1.4/216.1 MB 5.4 MB/s eta 0:00:41
   ---------------------------------------- 2.1/216.1 MB 7.2 MB/s eta 0:00:30
   ---------------------------------------- 2.6/216.1 MB 7.6 MB/s eta 0:00:29
    --------------------------------------- 3.5/216.1 MB 9.1 MB/s eta 0:00:24
    --------------------------------------- 4.0/216.1 MB 9.6 MB/s eta 0:00:23
    --------------------------------------- 4.8/216.1 MB 10.2 MB/s eta 0:00:21
   - -------------------------------------- 5.5/216.1 MB 11.1 MB/s eta


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Thanush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
#Installing torchvision
!pip install torchvision

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 660.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.7 MB 1.1 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.7 MB 1.3 MB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.7 MB 1.7 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.7 MB 2.6 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 3.8 MB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.7 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.7 MB 17.0 MB/s eta 0:00:01
   ------------------------ --------------- 1.6/2.7 MB 17.2 MB/s eta 0:00:01
   -------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Thanush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    
        self.fc2 = nn.Linear(64, 10)    
    def forward(self, x):
        x = F.relu(self.fc1(x))    
        return self.fc2(x)    

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

PyTorch Training time: 41.93 seconds
Test accuracy: 0.9680


## Convert PyTorch model to ONNX

In [ ]:
# Installing ONNX
!pip install onnx

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB 640.0 kB/s eta 0:00:25
   ---------------------------------------- 0.1/15.8 MB 825.8 kB/s eta 0:00:20
   ---------------------------------------- 0.1/15.8 MB 1.0 MB/s eta 0:00:16
    --------------------------------------- 0.2/15.8 MB 1.4 MB/s eta 0:00:11
   - -------------------------------------- 0.5/15.8 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.6/15.8 MB 2.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/15.8 MB 3.6 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/15.8 MB 4.6 MB/s eta 0:00:04
   ----- ---------------------------------- 2.2/15.8 MB 5.8 MB/s eta 0:00:03
   ------- -------------------------------- 3.0/15.8 MB 7.1 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/15.8 MB 8.1 MB/s eta 0:00:02
   ----------- ---------------------------- 4.5/15.8 MB 9.0 MB/s eta 0:00:02
  


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Thanush\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Loading and preprocessing the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Preparing datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.2949, Accuracy: 0.0938
Step 100, Loss: 0.8792, Accuracy: 0.7317
Step 200, Loss: 0.2799, Accuracy: 0.8012
Step 300, Loss: 0.1383, Accuracy: 0.8355
Step 400, Loss: 0.2365, Accuracy: 0.8536
Step 500, Loss: 0.0939, Accuracy: 0.8667
Step 600, Loss: 0.2381, Accuracy: 0.8762
Step 700, Loss: 0.1575, Accuracy: 0.8823
Step 800, Loss: 0.3133, Accuracy: 0.8872
Step 900, Loss: 0.1738, Accuracy: 0.8924
Step 1000, Loss: 0.1689, Accuracy: 0.8965
Step 1100, Loss: 0.2571, Accuracy: 0.9005
Step 1200, Loss: 0.1427, Accuracy: 0.9033
Step 1300, Loss: 0.1695, Accuracy: 0.9056
Step 1400, Loss: 0.4465, Accuracy: 0.9080
Step 1500, Loss: 0.3101, Accuracy: 0.9109
Step 1600, Loss: 0.0241, Accuracy: 0.9129
Step 1700, Loss: 0.1204, Accuracy: 0.9154
Step 1800, Loss: 0.0548, Accuracy: 0.9177
Training Accuracy for epoch 1: 0.9190

Epoch 2/5
Step 0, Loss: 0.0807, Accuracy: 0.9688
Step 100, Loss: 0.0358, Accuracy: 0.9570
Step 200, Loss: 0.1340, Accuracy: 0.9577
Step 300, Loss: 0.2204, Accuracy:

## Performance Optimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Loading and preprocessing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor
x_test = x_test / 255.0   # Normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Preparing datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     #Number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.4329, Accuracy: 0.1250
Step 100, Loss: 0.5306, Accuracy: 0.7296
Step 200, Loss: 0.3480, Accuracy: 0.8005
Step 300, Loss: 0.2732, Accuracy: 0.8326
Step 400, Loss: 0.1765, Accuracy: 0.8520
Step 500, Loss: 0.2427, Accuracy: 0.8645
Step 600, Loss: 0.4893, Accuracy: 0.8738
Step 700, Loss: 0.3610, Accuracy: 0.8799
Step 800, Loss: 0.1545, Accuracy: 0.8854
Step 900, Loss: 0.2645, Accuracy: 0.8896
Step 1000, Loss: 0.1534, Accuracy: 0.8941
Step 1100, Loss: 0.1284, Accuracy: 0.8975
Step 1200, Loss: 0.2175, Accuracy: 0.9007
Step 1300, Loss: 0.1207, Accuracy: 0.9026
Step 1400, Loss: 0.2155, Accuracy: 0.9048
Step 1500, Loss: 0.2218, Accuracy: 0.9074
Step 1600, Loss: 0.1198, Accuracy: 0.9098
Step 1700, Loss: 0.0646, Accuracy: 0.9119
Step 1800, Loss: 0.2575, Accuracy: 0.9144
Training Accuracy for epoch 1: 0.9159

Epoch 2/5
Step 0, Loss: 0.3035, Accuracy: 0.9375
Step 100, Loss: 0.1154, Accuracy: 0.9533
Step 200, Loss: 0.2005, Accuracy: 0.9521
Step 300, Loss: 0.1845, Accuracy: